In [ ]:
%matplotlib widget
from bmcs_matmod import GSM
import matplotlib.pylab as plt 
import sympy as sp
import numpy as np
from bmcs_matmod.gsm.potentials import Potential1DTEPDSymbExpr
sp.init_printing()

In [ ]:
p1d = Potential1DTEPDSymbExpr()

In [ ]:
gsm_F = GSM(
    u_vars = p1d.u_a,
    sig_vars = p1d.sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = (-1, 1, 1, -1),
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

In [ ]:
gsm_F.Eps_vars

In [ ]:
gsm_F.F_expr

In [ ]:
gsm_F.phi_

In [ ]:
sp.simplify(gsm_F.dF_dEps_)

In [ ]:
gsm_F.Sig_vars, gsm_F.Sig_

In [ ]:
sp.simplify(gsm_F.df_dSig_)

In [ ]:
sp.simplify(gsm_F.Phi_)[-1]

In [ ]:
sp.simplify(gsm_F.df_dEps_)

In [ ]:
gsm_F.dSig_dEps_

## Monotonic loading

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_KH_factor = 4
_KH = _E * _KH_factor
_K_ratio = 0.01 # 0.015
_K = _KH * _K_ratio
_H = _KH * (1 - _K_ratio)
material_params = dict(
    E_T_=_E, 
    gamma_T_= _H, # _E * 10, 
    K_T_= _K, # _E / 5,
    S_T_=0.008,
    c_T_=2.5,
    r_T_=2.7,
    f_s_=_f_s,
    X_0_=_X_0,  
    eta_T_=2000,
    T_0_=20,
    C_v_=0.01, # 0.0001, 
    beta_=0.0001,
    alpha_therm_=0, # 1.2e-5,
    d_N_ = 1
)

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI 

gsm_F.vp_on = True
gsm_F.update_at_k = False

def get_cyclic_load(max_s, max_t, n_cycles, n_incr):
    # Generating loading history
    s_arr = np.tile(np.array([-1, 1]), n_cycles) * np.linspace(0, max_s, 2 * n_cycles)
    s_arr = np.interp(np.linspace(0, max_t, n_incr * len(s_arr)), np.linspace(0, max_t, len(s_arr)), s_arr)

    # time array as input
    t_arr = np.linspace(0, max_t, len(s_arr))
    return s_arr, t_arr

In [ ]:
response_values = {}
max_s = 0.01
dot_s_list = [0.00005, 0.0005, 0.005]
for dot_s in dot_s_list:
#for dot_s in [0.01, 0.02, 0.05]: 
    print('dot_s', dot_s)
    u_t, t_t = get_cyclic_load( n_cycles=1, max_s=max_s, 
                                  max_t=max_s/dot_s, n_incr = 200)
    response_values[dot_s] = gsm_run(gsm_F, u_t[:,np.newaxis], 20 + 0 * t_t, t_t, **material_params)

In [ ]:
from scipy.integrate import cumtrapz
fig, ((ax, ax_T, ax_Diss), (ax_u_p, ax_omega, ax_4)) = plt.subplots(2,3, figsize=(12,6), tight_layout=True)

for dot_s, resp in response_values.items():
    
    label = f'$\dot{{s}}$ = {dot_s}'
    _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI = resp 
    _u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_F.Eps_as_blocks(_Eps_btI)
    _, _Z_atI, _X_atI, _Y_atI = gsm_F.Eps_as_blocks(_Sig_btI)
    ax.plot(_u_atI[0, :, 0], _sig_atI[0, :, 0], label=label);
    ax.legend()
    ax.set_title(r'stress-strain')
    ax.set_ylabel(r'$\varsigma$')
    ax.set_xlabel(r'$\varepsilon$')

    ax_T.plot(_u_atI[0, :, 0], _T_t, label=label);
    ax_T.legend()
    ax_T.set_title(r'temperature')
    ax_T.set_ylabel(r'$\vartheta$')
    ax_T.set_xlabel(r'$\varepsilon$')

    Diss_btI_F = cumtrapz(_dF_dEps_btI, _Eps_btI, initial=0, axis=1)
    ax_Diss.plot(_t_t, np.sum(Diss_btI_F[...,0], axis=0), alpha=1, label='F')
    ax_Diss.set_title(r'Dissipation')

    ax_omega.plot(_u_atI[0, :, 0], _omega_atI[0, :, 0])
    ax_omega.set_xlabel(r'$\varepsilon$/-')
    ax_omega.set_ylabel(r'$\omega$/-')
    ax_omega.set_title(r'Damage')

    ax_u_p.plot(_u_atI[0, :, 0], _Y_atI[0, :, 0])
    ax_u_p.set_xlabel(r'$\varepsilon$/-')
    ax_u_p.set_ylabel(r'$\varepsilon_\mathrm{p}$/-')
    ax_u_p.set_title(r'Plastic strain')
